In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib   

from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib

# Load data
eq5d = pd.read_csv('../rq1/rq1_cleaned_no_ae.csv')
scores = pd.read_excel('../data/Scores 6 Jan 2025_Prescribed_Completed Baseline PROMs.xlsx')

gad7 = scores[scores['promName']=='GAD7'][['SID','Round','total_score']]
gad7_wide = gad7.pivot_table(index='SID', columns='Round', values='total_score', aggfunc='first')
gad7_wide.columns = [f"GAD7_Round{r}" for r in gad7_wide.columns]
gad7_wide = gad7_wide.reset_index()
gad7 = pd.merge(eq5d, gad7_wide, on='SID', how='left')

ins = scores[scores['promName']=='insomniaEfficacyMeasure'][['SID','Round','total_score']]
ins_wide = ins.pivot_table(index='SID', columns='Round', values='total_score', aggfunc='first')
ins_wide.columns = [f"insomniaEfficacyMeasure_Round{r}" for r in ins_wide.columns]
ins_wide = ins_wide.reset_index()
full = pd.merge(gad7, ins_wide, on='SID', how='left')

full=full.drop(columns=['GAD7_Round1_x', 'insomniaEfficacyMeasure_Round1_x'])
full = full.rename(columns={'GAD7_Round1_y': 'GAD7_Round1', 'insomniaEfficacyMeasure_Round1_y': 'insomniaEfficacyMeasure_Round1'})

In [72]:
copy= full.copy()

In [73]:
drop_cols = [
    'SID', 'GAD7_Round2','GAD7_Round3','GAD7_Round4','GAD7_Round5','GAD7_Round6','GAD7_Round7',
    'GAD7_Round8','GAD7_Round9','GAD7_Round10','GAD7_Round11','GAD7_Round12',
    'GAD7_Round13', 'EQ5D_Round2','EQ5D_Round3','EQ5D_Round4','EQ5D_Round5',
    'EQ5D_Round6', 'insomniaEfficacyMeasure_Round2','insomniaEfficacyMeasure_Round3',
    'insomniaEfficacyMeasure_Round4','insomniaEfficacyMeasure_Round5',
    'insomniaEfficacyMeasure_Round6','insomniaEfficacyMeasure_Round7',
    'insomniaEfficacyMeasure_Round8','insomniaEfficacyMeasure_Round9',
    'insomniaEfficacyMeasure_Round10','insomniaEfficacyMeasure_Round11',
    'insomniaEfficacyMeasure_Round12','insomniaEfficacyMeasure_Round13'
]

In [74]:
cleaned_df = full.drop(columns=drop_cols).dropna()

In [75]:
cleaned_df.shape

(20577, 103)

In [76]:
df=cleaned_df.copy()

In [77]:
import pandas as pd
import numpy as np

# === Mappings ===
sex_map = {0: 'Male', 1: 'Female'}
smoking_map = {0: 'Never smoked', 1: 'Ex-smoker', 2: 'Current smoker'}
cannabis_map = {0: 'Never used', 1: 'Ex-user', 2: 'Current user'}
occupation_map = {0: 'Unemployed', 1: 'Employed', 2: 'Retired'}

# === Apply mappings ===
df['Sex'] = df['Sex'].replace(sex_map)
df['Smoking_status'] = df['Smoking_status'].replace(smoking_map)
df['Cannabis_status'] = df['Cannabis_status'].replace(cannabis_map)
df['occupation'] = df['occupation'].replace(occupation_map)

# === Add 'Total' group ===
df['Total'] = 'Total'
grouped = pd.concat([df, df.assign(Sex='Total')])

# === Summary functions ===
def mean_sd(series):
    return f"{series.mean():.1f} ({series.std():.1f})"

# === Continuous variable summary ===
summary = pd.DataFrame({
    'N': grouped.groupby('Sex').size(),
    'Age (mean (SD))': grouped.groupby('Sex')['Age'].apply(mean_sd),
    'Weight (mean (SD))': grouped.groupby('Sex')['weight'].apply(mean_sd),
    'Height (mean (SD))': grouped.groupby('Sex')['height'].apply(mean_sd),
    'Baseline EQ5D-5L (mean (SD))': grouped.groupby('Sex')['EQ5D_Round1'].apply(mean_sd),
    'Baseline GAD7 (mean (SD))': grouped.groupby('Sex')['GAD7_Round1'].apply(mean_sd),
    'Baseline Insomnia (mean (SD))': grouped.groupby('Sex')['insomniaEfficacyMeasure_Round1'].apply(mean_sd)
})

# === Occupation breakdown ===
occupation_counts = grouped.groupby(['Sex', 'occupation']).size().unstack(fill_value=0)
occupation_percents = occupation_counts.div(occupation_counts.sum(axis=1), axis=0).multiply(100).round(1)
occupation_str = occupation_counts.astype(str) + " (" + occupation_percents.astype(str) + "%)"
for col in occupation_str.columns:
    summary[f"{col}, n (%)"] = occupation_str[col]

# === Smoking status breakdown ===
smoking_counts = grouped.groupby(['Sex', 'Smoking_status']).size().unstack(fill_value=0)
smoking_percents = smoking_counts.div(smoking_counts.sum(axis=1), axis=0).multiply(100).round(1)
smoking_str = smoking_counts.astype(str) + " (" + smoking_percents.astype(str) + "%)"
for col in smoking_str.columns:
    summary[f"{col}, n (%)"] = smoking_str[col]

# === Cannabis status breakdown ===
cannabis_counts = grouped.groupby(['Sex', 'Cannabis_status']).size().unstack(fill_value=0)
cannabis_percents = cannabis_counts.div(cannabis_counts.sum(axis=1), axis=0).multiply(100).round(1)
cannabis_str = cannabis_counts.astype(str) + " (" + cannabis_percents.astype(str) + "%)"
for col in cannabis_str.columns:
    summary[f"{col}, n (%)"] = cannabis_str[col]

# === Final formatting ===
summary = summary.reset_index().rename(columns={'Sex': 'Group'})

# Optional: sort rows by Group order
group_order = ['Male', 'Female', 'Total']
summary['Group'] = pd.Categorical(summary['Group'], categories=group_order, ordered=True)
summary = summary.sort_values('Group')

# === Export or display ===
# summary.to_csv("baseline_summary.csv", index=False)
# summary.to_latex("baseline_summary.tex", index=False, escape=False)

# Show final table
print(summary)


    Group      N Age (mean (SD)) Weight (mean (SD)) Height (mean (SD))  \
1    Male  12429     40.5 (11.6)        86.7 (21.7)        178.1 (8.6)   
0  Female   8148     40.8 (13.3)        78.2 (22.9)        164.6 (7.8)   
2   Total  20577     40.6 (12.3)        83.3 (22.6)       172.8 (10.6)   

  Baseline EQ5D-5L (mean (SD)) Baseline GAD7 (mean (SD))  \
1                    0.4 (0.3)                12.2 (6.4)   
0                    0.3 (0.3)                12.1 (6.3)   
2                    0.4 (0.3)                12.2 (6.4)   

  Baseline Insomnia (mean (SD)) Employed, n (%) Retired, n (%)  \
1                     3.3 (2.1)    7962 (64.1%)     423 (3.4%)   
0                     3.3 (2.1)    4652 (57.1%)     394 (4.8%)   
2                     3.3 (2.1)   12614 (61.3%)     817 (4.0%)   

  Unemployed, n (%) Current smoker, n (%) Ex-smoker, n (%)  \
1      4044 (32.5%)          4357 (35.1%)     5545 (44.6%)   
0      3102 (38.1%)          2440 (29.9%)     3269 (40.1%)   
2      7146

In [44]:
full['Smoking_status'].value_counts()

Smoking_status
1    8821
2    6799
0    4973
Name: count, dtype: int64